# Onboard a model defined in a .wf1 file
This notebook takes a .wf1 workfile and transform  it to a modelflow model.<br>

The overall process is performed in a Dataclass named:**GrapWfModel**<br> 
Close study of this class can be rewarding, but is outside the need of most users.<br>
The overall structure is: 
 1. Eviews is started and the wf1 (or wf2) file is loaded
    -    Some transformations are performed on data.
    -    The model is unlinked 
    -    The workspace is saved as a wf2 file. Same name with ```_modelflow appended```
 5. Eviews is closed 
 6. The wf2 file is read as a json file. 
 7. Relevant objects are extracted. 
 7. The MFMSA variable is  extracted, to be saved in the dumpfile. 
 8. The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 9. Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)  
 9. For Stochastic equations new fitted variables are generated - without add add_factors and dummies.  
 9. A model to generate fitted variables is created  
 9. A model to generate add_factors is created. 
 9. A model encompassing the original equations, the model for fitted variables and for add_factors is created. 
 9. The data series and scalars are shoveled into a Pandas dataframe 
     - Some special series are generated as the expression can not be incorporated into modelflow model specifications
     - The model for fitted values is simulated in the specified timespan
     - The model for add_factors is simulated in the timespan set in MFMSA
 10. The data descriptions are extracted into a dictionary. 
    - Data descriptions for dummies, fixed values, fitted values and add_factors are derived. 
 11. Now we have a model and a dataframe with all variables which are needed.
 b
The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  


## Prerequisites  

- Eviews version 12 
- The python library: ```pyevies```  


In [1]:
%matplotlib Notebook

In [2]:
from pathlib import Path

from modelclass import model
from modelgrabwf2 import GrabWfModel
model.widescreen()

## Model specific transformations 
Not all Eviews equations have a direct counterpart in modelflow.<br>
To handle that two types of transformations can be provided for a specific model. 
- **eviews_run_lines** which specifies a list of Eviews command lines to be run. Typical to make some transformation of data which is not possible in modelflow. 
- **country_trans** which specifies a list of replacements to be done to the Eviews formulas before they are processed to modelflow. Typical to use the transformation mentioned above.  

In [3]:
filename = Path('paksolncons.wf1')
modelname = 'pak'

## Process the model

In [4]:
pak_trans = lambda input : input.replace('- 01*D(','-1*D(')                

all_about_pak = GrabWfModel(filename, 
                    country_trans = pak_trans,        
                    make_fitted = False,       # If we want to calculate a fittet value for stocastic eq 
                    do_add_factor_calc=True,   # Stocastic equations are argumented by add factor 
                    start = 2020,
                    end = 2100, 
                    fit_start = 2000,          # Start of calculation of fittet model in baseline 
                    fit_end   = 2030           # end of calc for fittted model, if None taken from mdmfsa options  
                    ) 


Reading M:\modelflow\modelflow-manual\papers\mfbook\content\EViews\paksolncons.wf1
Assummed model name: PAK
The model: PAK is unlinked 
Writing C:\Users\wb268970\OneDrive - WBG\Ldrive\MFM\modelflow\modelflow-manual\papers\mfbook\content\EViews\paksolncons_modelflow.wf2
Model name: PAK

Processing the model:PAK


Normalizing model: 100%|███████████████████████████████████████████████████████████████████████████████████████|303/303

Calculation of add factors for PAK calculated  


In [5]:
all_about_pak.mfmsa_options

''

## Check if each equation on its own result in the values provided. 
aka: residual check <br> 
If they are not pretty close, something is very wrong. 

In [28]:
all_about_pak.test_model(all_about_pak.start,
                         all_about_pak.end,
                         maxerr=100,tol=0.00001,
                         showall=0)

PAK calculated  

Chekking residuals for PAK 2020 to 2100

Variable with residuals above threshold
PAKNVCOLPRODKN              , Max difference:     0.00123121 Max Pct    0.0000045254% It is number    24 in the solveorder and error number 1
PAKNVCOLNIMPKN              , Max difference:     0.00026438 Max Pct    0.0000001358% It is number    25 in the solveorder and error number 2
PAKNVOILPRODKN              , Max difference:     0.01223154 Max Pct    0.0000045254% It is number    26 in the solveorder and error number 3
PAKNVOILNIMPKN              , Max difference:     0.00239856 Max Pct    0.0000001358% It is number    27 in the solveorder and error number 4
PAKNVGASPRODKN              , Max difference:     0.00753163 Max Pct    0.0000045254% It is number    28 in the solveorder and error number 5
PAKNVGASNIMPKN              , Max difference:     0.00009582 Max Pct    0.0000001358% It is number    29 in the solveorder and error number 6
PAKNVCOLNIMPCN              , Max difference:    

## Extract the model and the baseline
**all_about_ken** has a lot of content including. 
- .mmodel is the model instance
- .base_input is the baseline where the add factors and the fitted values are calculated 

In [29]:
#mpak,baseline     = all_about_pak()     
#baseline = all_about_pak.base_input
#mpak.model_description='Pakistan  model from paksolncons.wf1 27/3-2023'




# Save model info as pickle



In [30]:
eviews_dict = {v: f.eviews for v,f in all_about_pak.all_frml_dict.items()}
eviews_dict 

{'PAKNVINDMNFSKN': '@IDENTITY PAKNVINDMNFSKN  = PAKNVINDOTHRTKN  + PAKNVINDENGYSKN\r',
 'PAKNVINDMNFSCN': '@IDENTITY PAKNVINDMNFSCN  = PAKNVINDOTHRTCN  + PAKNVINDENGYSCN\r',
 'PAKNVINDMNFSXN': '@IDENTITY PAKNVINDMNFSXN  = PAKNVINDMNFSCN  / PAKNVINDMNFSKN\r',
 'PAKCRPSINTKN': '@IDENTITY PAKCRPSINTKN  = PAKBETAENGYCRPS  * PAKNVAGRCRPSKN  * (1  + ((PAKBETACRPS  + PAKBETAENGYCRPS)  / (1  - (PAKBETACRPS  + PAKBETAENGYCRPS))))\r',
 'PAKCRPSINTCN': '@IDENTITY PAKCRPSINTCN  = PAKBETAENGYCRPS  * PAKNVAGRCRPSCN  * (1  + ((PAKBETACRPS  + PAKBETAENGYCRPS)  / (1  - (PAKBETACRPS  + PAKBETAENGYCRPS))))\r',
 'PAKFSTYINTKN': '@IDENTITY PAKFSTYINTKN  = PAKBETAENGYFSTY  * PAKNVAGRFSTYKN  * (1  + ((PAKBETAFSTY  + PAKBETAENGYFSTY)  / (1  - (PAKBETAFSTY  + PAKBETAENGYFSTY))))\r',
 'PAKFSTYINTCN': '@IDENTITY PAKFSTYINTCN  = PAKBETAENGYFSTY  * PAKNVAGRFSTYCN  * (1  + ((PAKBETAFSTY  + PAKBETAENGYFSTY)  / (1  - (PAKBETAFSTY  + PAKBETAENGYFSTY))))\r',
 'PAKFSNGINTKN': '@IDENTITY PAKFSNGINTKN  = PAKBETAENGYFSNG  

## Run the model 
In order to achieve numerical stability Gauss-Seidle has to be dampened: alfa=0.7 makes sure that the solution does not explode. 
The convergence criteria is tightend a lot. 

In [32]:
import fnmatch
reverse_des  = {v:k for k,v in mpak.var_description.items()}
list_des = fnmatch.filter(reverse_des.keys(),'*GDP*')
list_var = [reverse_des[v] for v in list_des]


## And the simulation result is also fine. 
Here the percent difference is displayed

In [15]:
mpak['*GGEXPCAPTCN *NYGDPMKTPCN *GGDBTTOTLCN *BNCABFUNDCD PAKNVELECKN']

## Look at a stochastic variable 
Here the equations undergo more phases 

In [10]:
mpak.model_description

'Pakistan  model from paksolncons.wf1 27/3-2023'

## Now dump the model and data 

In [11]:
if 1:
    mpak.modeldump('models/270323_paksolncons.pcim')
    if 0: 
        try:
            mpal.modeldump('C:/modelflow manual/model_repo/270323_paksolncons.pcim')
        except: 
            print('could not write to modelrepo file')

In [12]:
mpakload,baselineload = model.modelload('models/270323_paksolncons.pcim',run = 1,alfa = 0.8)

file read:  C:\wb new\Charl\models\270323_paksolncons.pcim


In [13]:
xx = mpakload(baseline,2022,2050,alfa=0.7,silent=0)

Will start solving: PAK
Reusing the solver as no new data 
2022 Solved in 7 iterations
2023 Solved in 7 iterations
2024 Solved in 7 iterations
2025 Solved in 7 iterations
2026 Solved in 7 iterations
2027 Solved in 7 iterations
2028 Solved in 7 iterations
2029 Solved in 7 iterations
2030 Solved in 7 iterations
2031 Solved in 7 iterations
2032 Solved in 7 iterations
2033 Solved in 7 iterations
2034 Solved in 7 iterations
2035 Solved in 7 iterations
2036 Solved in 7 iterations
2037 Solved in 7 iterations
2038 Solved in 7 iterations
2039 Solved in 7 iterations
2040 Solved in 7 iterations
2041 Solved in 7 iterations
2042 Solved in 7 iterations
2043 Solved in 7 iterations
2044 Solved in 7 iterations
2045 Solved in 7 iterations
2046 Solved in 7 iterations
2047 Solved in 7 iterations
2048 Solved in 7 iterations
2049 Solved in 7 iterations
2050 Solved in 7 iterations
PAK solved  


In [1]:
mpak

NameError: name 'mpak' is not defined